# Data Warehouse Design
This notebook will outline the design of the data warehouse, including the star schema, fact and dimension tables.

In [18]:
# Importing files
import sqlite3
import os
import pandas as pd
import matplotlib.pyplot as plt

# Path to the database file
db_path = "Retail_dw.db"

## Creating Dimension Tables

In [19]:
# type: ignore
def connect_to_db():
    """Connect to the SQLite database"""
    conn = sqlite3.connect(db_path)
    print(f"Connected to database: {db_path}")
    return conn

def execute_query(query, params=None, fetch=True, commit=False):
    """Execute a query and optionally return results"""
    conn = connect_to_db()
    
    try:
        cursor = conn.cursor()
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        if commit:
            conn.commit()
            
        if fetch and cursor.description:
            results = cursor.fetchall()
            column_names = [description[0] for description in cursor.description]
            return results, column_names
    finally:
        conn.close()
        
    return None, None

def create_database_schema():
    """Create the database tables according to the schema"""
    conn = connect_to_db()
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Drop each table
    for table in tables:
        table_name = table[0]
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Dropped table: {table_name}")

    try:
        # Time Dimension Table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS Time_TB (
            TimeID INTEGER PRIMARY KEY,
            InvoiceDate TEXT,
            Quarter INTEGER,
            Year INTEGER,
            Month INTEGER

        )
        ''')
        
        # Product Dimension Table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS Product_TB (
            ProductID TEXT PRIMARY KEY,
            ProductName TEXT,
            Category TEXT
        )
        ''')
        
        # Customer Dimension Table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customer_TB (
            CustomerID TEXT PRIMARY KEY,
            CustomerName TEXT,
            Country TEXT
        )
        ''')

        
        # Fact Sales Table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS FactSales_TB (
            SaleID INTEGER PRIMARY KEY AUTOINCREMENT,
            TimeID INTEGER,
            ProductID INTEGER,
            CustomerID INTEGER,
            Quantity INTEGER,
            UnitPrice REAL,
            TotalSales REAL,
            FOREIGN KEY (TimeID) REFERENCES Time_TB (TimeID),
            FOREIGN KEY (ProductID) REFERENCES Product_TB (ProductID),
            FOREIGN KEY (CustomerID) REFERENCES Customer_TB (CustomerID)
        )
        ''')
        
        conn.commit()
        print("Database schema created successfully!")
        
    except sqlite3.Error as e:
        print(f"Error creating database schema: {e}")
    finally:
        conn.close()

## Check the tables 


In [20]:
# type: ignore
conn = sqlite3.connect("Retail_dw.db")
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", 
    conn
)
print(tables)
conn.close()

              name
0          Time_TB
1       Product_TB
2      Customer_TB
3         Store_TB
4     FactSales_TB
5  sqlite_sequence
